利用sklearn中的LatentDirichletAllocation库进行主题模型的学习

In [2]:
import jieba
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

#### 添加固定词和停用词

In [19]:
jieba.suggest_freq('沙瑞金', True)
jieba.suggest_freq('易学习', True)
jieba.suggest_freq('王大路', True)
jieba.suggest_freq('京州', True)

with open('../WordSegment/StopWords/stop_words_cn_1.txt', 'rb') as f:
    stoplist = f.read().decode('GBK').splitlines()
    
stoplist[:10]

[',', '?', '、', '。', '“', '”', '《', '》', '！', '，']

#### 语料的向量化

文本——>分词——>词频向量（由于LDA是基于词频统计的，因此一般不用TF-IDF来做文档特征）

In [21]:
# 载入文档
text_list = ['lda_text1.txt','lda_text2.txt','lda_text3.txt']
raw_texts = []
for item in text_list:
    path = './data/'+item
    with open(path, 'r') as f:
        raw_texts.append(f.read())
raw_texts

['沙瑞金赞叹易学习的胸怀，是金山的百姓有福，可是这件事对李达康的触动很大。易学习又回忆起他们三人分开的前一晚，大家一起喝酒话别，易学习被降职到道口县当县长，王大路下海经商，李达康连连赔礼道歉，觉得对不起大家，他最对不起的是王大路，就和易学习一起给王大路凑了5万块钱，王大路自己东挪西撮了5万块，开始下海经商。没想到后来王大路竟然做得风生水起。沙瑞金觉得他们三人，在困难时期还能以沫相助，很不容易。',
 '沙瑞金向毛娅打听他们家在京州的别墅，毛娅笑着说，王大路事业有成之后，要给欧阳菁和她公司的股权，她们没有要，王大路就在京州帝豪园买了三套别墅，可是李达康和易学习都不要，这些房子都在王大路的名下，欧阳菁好像去住过，毛娅不想去，她觉得房子太大很浪费，自己家住得就很踏实。',
 '347年（永和三年）三月，桓温兵至彭模（今四川彭山东南），留下参军周楚、孙盛看守辎重，自己亲率步兵直攻成都。同月，成汉将领李福袭击彭模，结果被孙盛等人击退；而桓温三战三胜，一直逼近成都。']

In [22]:
# 分词
texts = []
for text in raw_texts:
    texts.append(' '.join(jieba.lcut(text)))
texts

['沙瑞金 赞叹 易学习 的 胸怀 ， 是 金山 的 百姓 有福 ， 可是 这件 事对 李达康 的 触动 很大 。 易学习 又 回忆起 他们 三人 分开 的 前一晚 ， 大家 一起 喝酒 话别 ， 易学习 被 降职 到 道口 县当 县长 ， 王大路 下海经商 ， 李达康 连连 赔礼道歉 ， 觉得 对不起 大家 ， 他 最 对不起 的 是 王大路 ， 就 和 易学习 一起 给 王大路 凑 了 5 万块 钱 ， 王大路 自己 东挪西撮 了 5 万块 ， 开始 下海经商 。 没想到 后来 王大路 竟然 做 得 风生水 起 。 沙瑞金 觉得 他们 三人 ， 在 困难 时期 还 能 以沫 相助 ， 很 不 容易 。',
 '沙瑞金 向 毛娅 打听 他们 家 在 京州 的 别墅 ， 毛娅 笑 着 说 ， 王大路 事业有成 之后 ， 要 给 欧阳 菁 和 她 公司 的 股权 ， 她们 没有 要 ， 王大路 就 在 京州 帝豪园 买 了 三套 别墅 ， 可是 李达康 和 易学习 都 不要 ， 这些 房子 都 在 王大路 的 名下 ， 欧阳 菁 好像 去 住 过 ， 毛娅 不想 去 ， 她 觉得 房子 太大 很 浪费 ， 自己 家住 得 就 很 踏实 。',
 '347 年 （ 永和 三年 ） 三月 ， 桓 温兵 至 彭模 （ 今 四川 彭山 东南 ） ， 留下 参军 周楚 、 孙盛 看守 辎重 ， 自己 亲率 步兵 直攻 成都 。 同月 ， 成汉 将领 李福 袭击 彭模 ， 结果 被 孙盛 等 人 击退 ； 而桓温 三 战三胜 ， 一直 逼近 成都 。']

In [24]:
# 词频向量化
vector = CountVectorizer(stop_words=stoplist)
text_counter = vector.fit_transform(texts)
text_counter

<3x85 sparse matrix of type '<class 'numpy.int64'>'
	with 90 stored elements in Compressed Sparse Row format>

#### LDA模型

In [25]:
"""
LatentDirichletAllocation(n_components=10, doc_topic_prior=None,
                 topic_word_prior=None, learning_method='batch',
                 learning_decay=.7, learning_offset=10., max_iter=10,
                 batch_size=128, evaluate_every=-1, total_samples=1e6,
                 perp_tol=1e-1, mean_change_tol=1e-3, max_doc_update_iter=100,
                 n_jobs=None, verbose=0, random_state=None, n_topics=None)
n_components: LDA无监督学习的最重要超参数，主题数K
doc_topic_prior: 文档-主题先验狄利克雷分布参数
topic_word_prior: 主题-单词先验狄利克雷分布参数
learning_method: LDA的求解方法，batch为普通的变分推断EM算法，适用于样本量不大的学习；而online采用在线的批量学习，可用于大样本yield
learning_decay: online算法参数，控制收敛，取值最好在(0.5, 1.0]
learning_offset: online算法参数，控制偏置，取值要大于1
max_iter： EM算法最大迭代次数
total_samples： online算法参数，即每次学习的样本数量
batch_size：仅仅在算法使用"online"时有意义， 即每次EM算法迭代时使用的文档样本的数量。
mean_change_tol： EM算法，E步阈值
max_doc_update_iter： EM算法，E步阈值
"""

lda = LatentDirichletAllocation(n_topics=2)
result = lda.fit_transform(text_counter)

In [26]:
# 文档-主题向量
print(result)

[[0.98919502 0.01080498]
 [0.01689905 0.98310095]
 [0.98378971 0.01621029]]


In [27]:
# 主题-词分布
lda.components_

array([[1.49880433, 1.49880433, 2.4991167 , 2.4991167 , 2.4991167 ,
        0.5004542 , 1.49880433, 1.49880433, 2.4991167 , 0.5004542 ,
        1.49880433, 1.49913992, 0.5004542 , 0.5004542 , 1.49913992,
        0.50046643, 1.49880433, 1.49913992, 0.5004542 , 1.49880433,
        1.49913992, 0.50046643, 1.49913992, 1.49913992, 1.49913992,
        1.49880433, 1.49880433, 0.5004542 , 1.49880433, 1.49913992,
        1.49880433, 1.49913992, 1.49913992, 0.5004542 , 0.5004542 ,
        2.49877196, 0.5004542 , 1.49913992, 2.4991167 , 1.49880433,
        0.5004542 , 1.49880433, 2.49877196, 1.49913992, 1.49880433,
        2.49877196, 1.49880433, 0.50046643, 0.5004542 , 1.49913992,
        4.50639177, 1.49913992, 1.49880433, 2.50000278, 0.50046643,
        1.49880433, 0.500469  , 1.49880433, 2.50000278, 1.49913992,
        0.5004542 , 1.49880433, 5.49758907, 1.49880433, 1.49913992,
        1.49880433, 1.49913992, 1.49880433, 1.49880433, 0.5004542 ,
        1.49913992, 1.49880433, 2.50000278, 1.49